# Analysis of NJ public and charter schools results in ELA and math grades 6-8.

<span style="color: red;">**If kernel can't connect to server again run command:**
*netsh winsock reset*<span>

<a id="TOC"></a> 
## Table of Contents
1. [Data sources, definitions, about](#data)
2. [Imports: modules and data](#modules)
3. [Generating the map and its design elements](#maps) 


<a id="data"></a> 
## Data, definitions

#### Data:
1. Data New Jersey Student Learning Assessments (NJSLA) results 2015-2023 for grades 6-8 for public and charter schools:
<br>State of New Jersey, Department of Education:
Statewide Assessment Reports
<br>https://www.nj.gov/education/assessment/results/reports/
2. NJ schools locations: NJGIN Open Data <br>
https://njogis-newjersey.opendata.arcgis.com/datasets/d8223610010a4c3887cfb88b904545ff/explore

####  Performance levels for New Jersey Student Learning Standards for English Language Arts and Math  

**Level 1**: Did Not Yet Meet Expectations<br> 
**Level 2**: Partially Met Expectations <br>
**Level 3**: Approached Expectations  <br>
**Level 4**: Met Expectations  <br>
**Level 5**: Exceeded Expectations  <br>

*Source: New Jersey Assessments Resource Center, 2022, https://nj.mypearsonsupport.com/resources/reporting/NJSLA_Score_Interpretation_Guide_Spring2022.pdf*

## Questions
*1. How the test results changed?*
<br>Changes in test scores proportions are charted for MATH and ELA for years 2015-2023 for middle school grades (grades 6-8).
<br><br>
*2. What are academic results of the school?* 
<br> The schools are compared by the sum of average level 5 scores for years 2015-2023 for all middle grades combined (or the years available in NJ DOE data for part of these years).

## Limitations
1. Some elementary schools go up to grade 6. For these schools' share of level 5 results is usually higher than in schools with grades 6-8 or 7-8. Since they teach only the first of the middle grades, they were excluded to make a more grounded view of the middle schools quality.<br><br>
2. Some school names in the original NJSLA data are inconsistently spelled or contain errors in the records across different years. As a result, these discrepancies created separate entries in the allResultsAVG2015_23DF dataframe. Consequently, certain schools have multiple overlapping points on the map, with pop-ups displaying data for different years.
While this may affect the map's visual clarity and completeness, the current representation still provides a comprehensive overview of the academic proficiency of middle schools in New Jersey. Further data cleaning to eliminate this issue required more time and effort, which was unnecessary for the project's purpose.

#### About this notebook

- The notebook '*1._Data_processing_by_NJ_middle_schools*' contains the steps for processing data on state testing of public and charter schools in New Jersey. 
- This notebook '*2._Generating_map_by_NJ_middle_schools*' contains code to generate the map from the processed data.
- The map is available at: https://njmsmap.netlify.app

<a id="modules"></a> 
#### Imports: modules

In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
# from shapely.geometry import Point
from tqdm import tqdm

pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
basePath = r"G:\My Drive\Kids\NJ_schools_mapped"
dataFolder = r"raw_data"
outputFolder = r"processed_data"

#### Read schools geolocation file

In [3]:
# Read saved GeoJSON with average tests results and plots
schoolsFile = 'NJpublicSchoolsData.geojson'
schoolsPath = os.path.join(basePath, outputFolder,schoolsFile)
schoolsData = gpd.read_file(schoolsPath)

In [4]:
# Checking the geodataframe
schoolsData.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   OBJECTID                 1025 non-null   int64   
 1   DIST_NAME                1025 non-null   object  
 2   SCHOOLTYPE               1008 non-null   object  
 3   SCHOOL                   1025 non-null   object  
 4   SCHOOLNAME               1025 non-null   object  
 5   CITY                     1025 non-null   object  
 6   School_Key               1025 non-null   object  
 7   School Name              1025 non-null   object  
 8   8yrs avg Lvl 1 Math      1021 non-null   float64 
 9   8yrs avg Lvl 2 Math      1021 non-null   float64 
 10  8yrs avg Lvl 3 Math      1021 non-null   float64 
 11  8yrs avg Lvl 4 Math      1021 non-null   float64 
 12  8yrs avg Lvl 5 Math      1021 non-null   float64 
 13  8yrs avg Lvl 1 ELA       1022 non-null   float64 
 14  

<a id="maps"></a> 
### Map of the NJ middle schools by assessment results 

Below mapping is done with the __[Folium library](https://python-visualization.github.io/folium/latest/)__

#### Producing map of New Jersey middle schools

In [5]:
# Prepare legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="flex-grow: 1; margin-bottom: 10px;">
                   Circle size - the sum of average shares of students scoring at the highest level (level 5) on New Jersey state ELA and math tests over the years 2015 to 2023.
                   </div>                   
                   <div><i class="fa fa-circle" style="border:2px solid #06a6cf; color:white; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">public school </span></div>
                   <br>
                   <div><i class="fa fa-circle" style="border:2px solid #f0a607; color:white; border-radius:50%; display:inline-block;"></i><span style="margin-left: 5px;">charter school </span></div>
                   
      </div>
     '''

In [11]:
# Add map introduction

intro_html = """
     <div style="position: fixed; 
                 top: 50px; left: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="margin-bottom: 10px;">
                       <span style = "font-weight:bold; font-size: 1.5em;">New Jersey Public and Charter Middle Schools
                       </span>
                   </div>  
                   <div style="flex-grow: 1;">The map is part of the project to map middle schools academic results in New York and New Jersey. See more:
                   <br>
                   <ul>
                       <li><a href = "https://github.com/Solirinai/NY_schools_maps" target="_blank">Github repository with code and data sources.</a></li>
                       <li><a href = "https://nycmsmap.netlify.app" target="_blank">Map of New York City public middle schools.</a></li>
                       <li><a href = "https://nysmsmap.netlify.app" target="_blank">Map of New York State public and charter middle schools.</a></li>
                   </ul>
                   </div>                 
                 
      </div>
"""

In [6]:
# Add click instructions

intro_click = '''<div style="position: fixed; 
                 top: 50px; right: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="flex-grow: 1;">
                    Click on a circle to see historical test results for middle school grades (grades 6-8) for the school.
                   </div>                                     
      </div>
     '''

In [13]:
# Add click instructions

notes = '''<div style="position: fixed; 
                 bottom: 50px; right: 50px; width: 300px;  
                 border:0px solid grey; z-index:9999; font-size:10px;
                 background-color: rgba(255, 255, 255, 0.7);
                 padding: 10px;
                 display: flex;
                 flex-direction: column;
                 max-height: calc(100vh - 100px); /* Prevent overflow */
                 overflow-y: auto; /* Add scrollbar if content exceeds max-height */                 
                 ">
                   <div style="margin-bottom: 10px;">
                   <span style = "font-weight:bold; font-size: 1.0em;">Limitations:</span>
                   </div> 
                   <div style="flex-grow: 1; margin-bottom: 10px;">
                       <div>
                       <ul>
                           <li> K-6 schools are excluded.</li>
                           <li> Due to inconsistent spelling of school names in the source data, certain schools have multiple overlapping circles on the map, with pop-ups displaying data for different years.</li>
                       </ul>
                       </div> 
                       <div>
                       <span style = "font-weight:bold; font-size: 1.0em;">Data</span>
                       <ul>
                       <li><a href = 'https://www.nj.gov/education/assessment/results/reports/' target="_blank">New Jersey Statewide Assessment Reports, State of New Jersey, Department of Education</a>.</li>
                       <li><a href = "https://njogis-newjersey.opendata.arcgis.com" target="_blank"> NJ schools locations: NJGIN Open Data</a>.</li>
                       </ul> 
                       </div>
                       <div>Map developed by <a href = "https://github.com/Solirinai" target="_blank">Solirinai</a>.</div>
                   </div>
      </div>
     '''

In [14]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

# Create a map object, centered at New Jersey

schools_map = folium.Map(location=[40.0842, -74.5357], zoom_start=9, tiles="cartodb positron")
   
# Add function to make size of the circle marker proportional to the '8yrs avg Lvl 5 Math+Ela' column

def my_style(x):
    level5 = x['properties']['8yrs avg Lvl 5 Math+Ela']
    charter_alt = x['properties']['School Name']
    color = '#f0a607' if 'CHARTER' in (charter_alt or '') else '#06a6cf' 
    if level5 is None:
        level5 = 0
    return {
        "radius": (level5)*2000,
        "color": color        
    }  

# Function to create iframe for a given row
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br>\
    <strong>{2}:</strong> {3}<br>\
    <strong>{4}:</strong> {5}<br>\
    <strong>{6}:</strong> {7}<br>\
    <strong>{8}:</strong> {9}<br>\
    <img src="data:image/png;base64,{10}"><br>\
    <img src="data:image/png;base64,{11}">'.format(
        'School Name', row['SCHOOLNAME'],
        'School District', row['DIST_NAME'],
        'School Name from NJ DOE data', row['School_Key'],
        'Level 5 share Avg 2015-2023 Math', round(row['8yrs avg Lvl 5 Math'], 2), 
        'Level 5 share Avg 2015-2023 ELA', round(row['8yrs avg Lvl 5 ELA'], 2),
        row['plot Math'], row['plot ELA'])
    return folium.IFrame(html, width=500, height=450)

def create_popup(x):
    iframe = create_iframe(x)
    popup = folium.Popup(iframe)
    return popup

# Iterate over the geodataframe and add a pop-up to each feature
for _, row in tqdm(schoolsData.iterrows(), total = len(schoolsData)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=schoolsData.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='green', fill_opacity=0, color="green", weight=2),
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False,       
).add_to(schools_map)    
        
# Adding HTML elements to the map:

schools_map.get_root().html.add_child(folium.Element(legend_html))
schools_map.get_root().html.add_child(folium.Element(intro_html))
schools_map.get_root().html.add_child(folium.Element(intro_click))
schools_map.get_root().html.add_child(folium.Element(notes))
  
# # Display the map in the notebook (not recommended due to big size of the map)
# schools_map

# Save map to html
mfile = 'NJMidSchoolsMap.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving map to file {mpath} ...')
schools_map.save(mpath)
print("Saved.")     

del mpath, mfile

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [00:16<00:00, 62.61it/s]


Saving map to file G:\My Drive\Kids\NJ_schools_mapped\processed_data\NJMidSchoolsMap.html ...
Saved.
